In [1]:
!pip install unrar
!pip install rarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import re
import rarfile
import zipfile
import codecs
from sklearn.model_selection import train_test_split

In [3]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPool1D, Dropout, MaxPooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
! gdown --id 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg
To: /content/hamshahri.rar
100% 873M/873M [00:07<00:00, 112MB/s]


In [5]:
def extract_rar_file(path, extract_path):
  files = []
  with rarfile.RarFile(path, 'r') as rf:
    files = rf.namelist()
    rf.extractall(extract_path)
    
  for file in files:
    path_to_file = fr"{extract_path}\{file}"  

In [6]:
path = 'hamshahri.rar'
extract_path = 'hamshahri'
extract_rar_file(path, extract_path)

In [13]:
def extract_zip_file(path, extract_path):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

In [8]:
hamshahri_corpus_path = 'hamshahri/hamshahriold/Corpus/Hamshahri-Corpus.zip'
hamshahri_corpus_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus'
extract_zip_file(hamshahri_corpus_path, hamshahri_corpus_extract_path)

In [9]:
HAMSHAHRI_CORPUS_PATH = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus/Hamshahri-Corpus.txt'

In [10]:
%%time
with codecs.open(HAMSHAHRI_CORPUS_PATH, "r", "utf-8") as text:
    sentences = []
    pattern = re.compile(".Cat")
    s = ' '
    for line in text:
      for match in re.finditer(pattern, line):
        sentences.append((line,s))
        s = ' '
      s += line         

CPU times: user 57.6 s, sys: 2.17 s, total: 59.8 s
Wall time: 57.6 s


In [11]:
category_punctuation = [re.sub('\.Cat\t', '', line[0]) for line in sentences]
sentences_punctuation = [line[1].replace('\n', '') for line in sentences] 
sentences_punctuation = [line.replace('\r', '') for line in sentences_punctuation] 
sentences_punctuation = [re.sub('\.Cat\s\w+', '', sentence) for sentence in sentences_punctuation]
sentences_punctuation = [re.sub('\.DID\\t\d+S\d+.Date\\t\d+\\\\\d+\\\\\d+', '', sentence) for sentence in sentences_punctuation]

In [12]:
print(sentences_punctuation[1])

  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين ن

In [13]:
corpus = []
for i in range(len(sentences)):
  corpus.append({'text': sentences_punctuation[i], 'category': category_punctuation[i-1]})

In [14]:
corpus[1]

{'category': 'adabh\r\n',
 'text': '  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پاي

In [15]:
df = pd.DataFrame(corpus[1:], columns=['text', 'category'])

In [16]:
df.head()

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n
3,را جدي بگيريم مطالعه اي مقدماتي پيرامون نقش ...,ejtem\r\n
4,سياه و غباري كه سترده نمي شود... اشاره; به ر...,elmfa\r\n


In [17]:
len(df)

165261

### **Export DataFrame to CSV**

In [18]:
df.to_csv('data.csv')

# **Read Dataset**

In [4]:
data = pd.read_csv('data.csv')

In [5]:
data = data.iloc[: , 1:]

In [6]:
data.head(3)

,text,category
0,در زندگي گروهي از طريق هنر نگاهي به نمايشگاه...,adabh\r\n
1,هنري جهان نمايشگاه هنر در خدمت ديكتاتورها نم...,adabh\r\n
2,نگارخانه ها گالري گلستان: نمايشگاه طرح و ساخ...,adabh\r\n


In [7]:
data['category'].nunique()

142

In [8]:
len(data)

165261

In [9]:
data = data[data.groupby('category')['category'].transform('count') > 1].copy()

In [10]:
len(data['category'].unique())

105

In [11]:
data['category'].value_counts()

siasi\r\n      17530
eqtes\r\n      16661
kharj\r\n      14666
akhar\r\n      13811
vrzsh\r\n      13010
               ...  
Musical\r\n        3
igozar\r\n         3
Cinew\r\n          3
Polig\r\n          2
Thought\r\n        2
Name: category, Length: 105, dtype: int64

### **Read StopWords**

In [14]:
hamshahri_stopwords_path = 'hamshahri/hamshahriold/Corpus/PersianStopWords.zip'
hamshahri_stopwords_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-stopwords'
extract_zip_file(hamshahri_stopwords_path, hamshahri_stopwords_extract_path)
HAMSHAHRI_STOPWORDS_PATH = '/content/hamshahri/hamshahriold/Corpus/hamshahri-stopwords/PersianStopWords.txt'

In [15]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [16]:
stopwords = read_stop_words(HAMSHAHRI_STOPWORDS_PATH)
print(len(stopwords))

796


In [17]:
# Convert pandas to numpy
np_data = data["text"].values

In [18]:
np_data[0]

'  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين 

**Install hazm library**

In [19]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
word_tokenized = [word_tokenize(t) for t in np_data]

In [21]:
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [22]:
del word_tokenized
del np_data
del df
del corpus

NameError: ignored

In [23]:
corpus = []
for sentence in word_tokenized_filtered:
  s = ' '
  for w in sentence:
    s += w + ' '
  corpus.append(s)  

In [24]:
print(corpus[0])

 زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان باطن ظاهري انسانهاهستند هيبتي حشره تابلوهاي نقاشي نمايشگر گوشه زندگي مورچه ديده ايم سنگين خودرا دوش كشد راه خسته نتيجه آمدن حادثه اتفاقي دستش رها گريز خطر گذارد فرار ساعتي نقطه برمي گردد دوش كشيده نفس زنان عرق ريزان منزل رساند داند راه رفت مورچه استعداد غريزه عملي شروع پايان رساند داند نسل آينده مزاياي استفاده نمايد داستان درازي انتها پاياني ميليونها سال گذشته ميلياردها سال خواهد_گذشت وهمين برنامه ميليونها تجديد گفت سير حيات تكامل پايان ناپذير اسرار جهان پاياني بخواهيم چيز پايان ناپذير برسيم انتهايي نيست جايي رسيم كتاب مورچگان نوشته موريس مترلينگ مورچگان تابلوهاي نقاشي احمد طباطبايي جايگاه ويژه دارند نمايشگاهي آثار هنرمند اول 23 خرداد نگارخانه برگ برپا 36 اثر موضوع نمايش عمومي درآمد طباطبايي مدت 18 سالي نقاشي پردازد موضوع مورچگان كار علت انتخاب سوژه گويد روزهاي زندگي غمگين بودم زماني خواب رفتم مورچه ديدم تصميم گرفتم طرح بكشم كار نظرم جالب آمد روز طراحي ادامه دادم نقاشي مدلها تغييراتي ايجاد ضمن حشره

**category tokenizaer**

In [25]:
category_tokenizer = Tokenizer()
category_tokenizer.fit_on_texts(list(data['category']))
category_data = category_tokenizer.texts_to_sequences(list(data['category']))

In [26]:
category_data

[[12],
 [12],
 [12],
 [9],
 [14],
 [14],
 [14],
 [14],
 [14],
 [14],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [16],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [15],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [12],
 [9],
 [9],
 [14],
 [14],
 [14],
 [14],
 [14],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [16],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [13],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [15],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [8],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [1

In [27]:
num_categories = len(category_tokenizer.word_index)+1

In [28]:
num_categories

83

In [29]:
category_tokenizer.word_index

{'abksh\r': 33,
 'adabh\r': 12,
 'adarman\r': 71,
 'aeqts\r': 18,
 'akhar\r': 4,
 'art\r': 60,
 'artw\r': 25,
 'axrooz\r': 36,
 'bankb\r': 28,
 'bazar\r': 43,
 'books\r': 47,
 'busiw\r': 67,
 'cartoon\r': 56,
 'cinama\r': 69,
 'cinew\r': 59,
 'city\r': 52,
 'cultw\r': 24,
 'donya\r': 74,
 'earth\r': 53,
 'econo\r': 41,
 'econw\r': 22,
 'ejtem\r': 9,
 'elmfa\r': 14,
 'elmif\r': 7,
 'eqtes\r': 2,
 'eqtsj\r': 42,
 'ertebat\r': 79,
 'erteg\r': 75,
 'gards\r': 39,
 'globa\r': 54,
 'gofgu\r': 77,
 'goftg\r': 81,
 'gozar\r': 16,
 'gqarn\r': 80,
 'gungn\r': 11,
 'hamln\r': 63,
 'havad\r': 13,
 'igozar\r': 82,
 'ikabar\r': 64,
 'imaqal\r': 76,
 'infor\r': 55,
 'intep\r': 27,
 'jjahn\r': 48,
 'jvarz\r': 21,
 'kharj\r': 3,
 'lastp\r': 46,
 'lifew\r': 34,
 'lite\r': 73,
 'maqal\r': 15,
 'maref\r': 32,
 'media\r': 70,
 'mohit\r': 26,
 'mskan\r': 66,
 'musical\r': 61,
 'nameh\r': 29,
 'newsp\r': 35,
 'nnaft\r': 20,
 'norooz\r': 78,
 'polig\r': 17,
 'sanat\r': 40,
 'santj\r': 31,
 'scien\r': 57,
 'sc

In [30]:
Y = [to_categorical(category, num_categories) for category in category_data]

In [31]:
Y[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]], dtype=float32)

In [32]:
del data

**text tokenizer**

In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [34]:
vocab_size = len(tokenizer.word_index)+1

In [35]:
data_to_sequences = tokenizer.texts_to_sequences(corpus)

In [36]:
maxlen = 500
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen = maxlen)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(pad_data_to_sequences, Y, test_size=0.3, random_state=42)

In [44]:
del data_to_sequences
del pad_data_to_sequences

### **Neural Network**

In [38]:
embedding_dim = 50

In [49]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = maxlen))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(num_categories, activation='softmax'))

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           26069400  
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dense_1 (Dense)             (None, 83)                8383      
                                                                 
Total params: 26,138,183
Trainable params: 26,138,183
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

ValueError: ignored

In [50]:
len(Y)

165224

In [51]:
len(X_train)

115656

In [52]:
len(X_test)

49568